In [34]:
import os

from dotenv import load_dotenv

from langchain.chains import RetrievalQA
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.prompts import PromptTemplate

load_dotenv()

True

In [31]:
loader = CSVLoader(file_path="anticorrupcion_intel.csv", encoding="utf-8", csv_args={'delimiter': ','})
data = loader.load()

In [ ]:
text_splitter = CharacterTextSplitter(separator='\n')
splits = text_splitter.split_documents(data)
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))
vectordb = DocArrayInMemorySearch.from_documents(splits, embeddings)
retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 2, "fetch_k": 4})


In [35]:
# Filter the movies based on the user input
#df_filtered = md[md['genres'].apply(lambda x: genre in x)]

template_prefix = """Tu eres un investigador de corrupción y quieres encontrar patrones en los datos. 
Usa el siguiente contexto para responder la pregunta que se encuentra al final. 
En caso de no sber la respuesta, contesta con "No sé la respuesta". No busques información adicional.

{context}"""
user_info = """Esta es la información que sabes para encontrar patrones, usa esta infrmación para mejorar tu búsqueda:
nivelSalarial: {nivelsalarial}
Genero: {genero}"""

template_suffix= """Pregunta: {question}
Tu respuesta:"""

user_info = user_info.format(nivelsalarial = 5, genero = 0)

COMBINED_PROMPT = template_prefix +'\n'+ user_info +'\n'+ template_suffix
print(COMBINED_PROMPT)

#setting up the chain
llm = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

PROMPT = PromptTemplate(
    template=COMBINED_PROMPT, input_variables=["context", "question"])

chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True,
                                 chain_type_kwargs=chain_type_kwargs)




Tu eres un investigador de corrupción y quieres encontrar patrones en los datos. 
Usa el siguiente contexto para responder la pregunta que se encuentra al final. 
En caso de no sber la respuesta, contesta con "No sé la respuesta". No busques información adicional.

{context}
Esta es la información que sabes para encontrar patrones, usa esta infrmación para mejorar tu búsqueda:
nivelSalarial: 5
Genero: 0
Pregunta: {question}
Tu respuesta:


In [37]:
query = '¿Qué políticas públicas recomiendas implementar para reducir la corrupción?'
if query:
    result = qa({"query": query})
    print(result['result'])

C:\Users\jesla\AppData\Local\Temp\ipykernel_19648\2706562065.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"query": query})


Para reducir la corrupción, recomendaría implementar las siguientes políticas públicas:

1. Transparencia y acceso a la información: Es fundamental que la información sobre el uso de los recursos públicos sea accesible y comprensible para todos los ciudadanos. Esto incluye información sobre contratos, licitaciones, salarios y otros gastos del gobierno.

2. Auditorías y controles internos: Es necesario fortalecer los sistemas de auditoría y control interno en todas las instituciones públicas para detectar y prevenir actos de corrupción.

3. Sanciones efectivas: Las sanciones por actos de corrupción deben ser severas y efectivas para disuadir a los funcionarios de participar en estas prácticas.

4. Educación y cultura de la legalidad: Es importante fomentar una cultura de la legalidad y la ética en la sociedad, lo cual incluye la educación sobre los efectos negativos de la corrupción.

5. Participación ciudadana: Los ciudadanos deben tener un papel activo en la supervisión del gobierno y